In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from secret import AK_SK
keys = AK_SK('../qianfan.keys')
os.environ["QIANFAN_ACCESS_KEY"] = keys.get_ak()
os.environ["QIANFAN_SECRET_KEY"] = keys.get_sk()


In [3]:
from myqianfan import QianfanLLM
from myqianfan import model_spec

llm = QianfanLLM(model_spec=model_spec.Tiny8K, temperature=0.3)

print(llm.invoke("你好, 世界上最大的动物是什么？"))


[WARNING][2024-10-14 22:37:10.648] redis_rate_limiter.py:21 [t:8703163072]: no redis installed, RedisRateLimiter unavailable
[WARNING][2024-10-14 22:37:11.119] base.py:916 [t:8703163072]: This key `disable_search` does not seem to be a parameter that the model `ERNIE-Tiny-8K` will accept


世界上最大的动物是鲸鱼。鲸鱼是一种大型海洋哺乳动物，属于鲸目。它们生活在海洋中，体型巨大，是已知动物中最大的种类之一。


In [24]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://aitutor.liduos.com/01-llm/01-1.html",
     "https://aitutor.liduos.com/01-llm/01-2.html",
     "https://aitutor.liduos.com/01-llm/01-3.html"),
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content", "post-title", "post-header")
    #     )
    # ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [28]:
splits[9].page_content + "\n===\n" + splits[10].page_content

'LangChain入门\n            \n                \n\n\n\n            \n                    \n                    LangChain介绍\n            \n                \n\n\n\n            \n                    \n                    LangChain模块学习\n===\nLangChain介绍\n            \n                \n\n\n\n            \n                    \n                    LangChain模块学习'

In [30]:
from myqianfan import QianfanEmbedding
vectorstore = Chroma("qianfan", embedding_function=QianfanEmbedding(), persist_directory="vectorstore/qianfan.vec")
batch_size = 16
doc_ids = []
for i in range(0, len(splits), batch_size):
    print(f"Adding documents {i} to {i+batch_size}")
    texts = [doc.page_content for doc in splits[i:i+batch_size]]
    metadatas = [doc.metadata for doc in splits[i:i+batch_size]]
    doc_ids += vectorstore.add_texts(texts, metadatas)


Adding documents 0 to 16
Adding documents 16 to 32
Adding documents 32 to 48
Adding documents 48 to 64
Adding documents 64 to 80
Adding documents 80 to 96
Adding documents 96 to 112
Adding documents 112 to 128
Adding documents 128 to 144
Adding documents 144 to 160
Adding documents 160 to 176
Adding documents 176 to 192
Adding documents 192 to 208
Adding documents 208 to 224
Adding documents 224 to 240
Adding documents 240 to 256
Adding documents 256 to 272
Adding documents 272 to 288
Adding documents 288 to 304
Adding documents 304 to 320


In [31]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("大语言模型的应用场景有哪些？")

/Users/flybywindwen/miniconda3/envs/llm-wechat/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
[WARNING][2024-10-14 23:40:08.501] base.py:916 [t:8703163072]: This key `disable_search` does not seem to be a parameter that the model `ERNIE-Tiny-8K` will accept


'大语言模型的应用场景包括但不限于以下几个方面：\n\n1. 自然语言处理：用于文本生成、情感分析、机器翻译等任务。\n2. 智能客服：能够提供智能化的客户服务，帮助回答用户的问题和提供解决方案。\n3. 智能推荐系统：能够根据用户的兴趣和行为，提供个性化的产品和服务推荐。\n4. 智能语音助手：可以应用于智能手机、智能家居等设备，帮助用户实现语音控制。\n5. 科研领域：用于数据挖掘、机器学习、自然语言处理等领域的研究和应用。\n\n总的来说，大语言模型是一种强大的技术工具，具有广泛的应用前景。'

In [32]:
rag_chain.invoke("掩码语言模型")

[WARNING][2024-10-14 23:41:54.851] base.py:916 [t:8703163072]: This key `disable_search` does not seem to be a parameter that the model `ERNIE-Tiny-8K` will accept


'掩码语言模型是一种训练方法，用于对输入文本中的词元进行掩码，基于上下文来预测被掩码的词元。BERT 是典型的掩码语言模型之一，具有在许多 NLP 任务中取得最佳表现的能力。它使用 Transformer 架构等技术，在大型文本语料上训练，并在自然语言处理领域中扮演重要角色。著名的掩码语言模型包括 BERT、RoBERTa 和 T5 等。这些模型在早期发展阶段需要基于具体下游任务的数据集进行微调。在 LLM 的早期发展阶段，BERT 主要用于自然语言理解任务，通过双向预训练语言模型和微调技术来提高模型的性能。'

In [33]:
rag_chain.invoke("使用OpenAI需要准备什么")

[WARNING][2024-10-14 23:44:17.823] base.py:916 [t:8703163072]: This key `disable_search` does not seem to be a parameter that the model `ERNIE-Tiny-8K` will accept


'使用OpenAI需要准备一些资料和工具，包括相关的许可证、开发环境、API密钥等，以及相关的技术文档和参考资料。具体来说，需要了解OpenAI的相关API和SDK，熟悉开发环境和技术要求，掌握相关的编程语言和工具，以及了解相关的安全性和隐私保护措施。同时，还需要对相关的法律法规和政策要求进行了解和遵守。'

In [34]:
llm.conversation_history

[{'role': 'user', 'content': '你好, 世界上最大的动物是什么？'},
 {'role': 'assistant',
  'content': '世界上最大的动物是鲸鱼。鲸鱼是一种大型海洋哺乳动物，属于鲸目。它们生活在海洋中，体型巨大，是已知动物中最大的种类之一。'},
 {'role': 'user',
  'content': "Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: 大语言模型的应用场景有哪些？ \nContext: 大语言模型概述\n            \n                \n\n\n\n            \n                    \n                    大语言模型概况\n            \n                \n\n\n\n            \n                    \n                    你好, ChatGPT\n\n大语言模型概述\n            \n                \n\n\n\n            \n                    \n                    大语言模型概况\n            \n                \n\n\n\n            \n                    \n                    你好, ChatGPT\n\n大语言模型概述\n            \n                \n\n\n\n            \n                    \n           

In [37]:
rag_chain.invoke("Chat Completions 会话补全和响应")

[WARNING][2024-10-14 23:49:33.749] base.py:916 [t:8703163072]: This key `disable_search` does not seem to be a parameter that the model `ERNIE-Tiny-8K` will accept


'根据提供的Context，Chat Completions会话补全是使用频次最高的接口之一，几乎所有套壳ChatGPT应用都基于这个接口。给定一组描述对话的消息列表，模型将返回一个回复，其中包含关于Chat Completions会话补全的详细信息。\n\n在给出的Context中，模型返回了一个关于ChatGPT的回复，并提供了OpenAI文档解读。你可以通过调用OpenAI API创建聊天机器人，并使用提供的消息列表来获取相应的回复。\n\n至于如何动手实现聊天机器人，这取决于你的具体需求和技能水平。你可以参考OpenAI的文档或相关教程，学习如何使用OpenAI API创建聊天机器人。同时，你也可以根据自己的创意和想法，编写相应的代码来实现聊天机器人。'